## Install pyTigerGraph

In [ ]:
# Setup
!pip install -U pyTigerGraph

## Add Imports and Establish Initial Connection

In [ ]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters
hostName = ''
userName = ''
password = ''

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print('Connected')

## Define and Publish the Schema

In [ ]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql("""
  USE GLOBAL
  CREATE VERTEX BusService (PRIMARY_ID name STRING) WITH primary_id_as_attribute="true"
  CREATE VERTEX BusStop (PRIMARY_ID id UINT, name STRING, latitude FLOAT, longitude FLOAT) WITH primary_id_as_attribute="true"
  CREATE VERTEX TrainService (PRIMARY_ID name STRING) WITH primary_id_as_attribute="true"
  CREATE VERTEX TrainStation (PRIMARY_ID code STRING, name STRING, latitude FLOAT, longitude FLOAT) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE serve (From BusService, To BusStop | From TrainService, To TrainStation, service_seq_no UINT)
  
  CREATE VERTEX Location (PRIMARY_ID id UINT, address STRING, latitude FLOAT, longitude FLOAT)
  CREATE VERTEX Trip (PRIMARY_ID id UINT, start_time DATETIME, end_time DATETIME) WITH primary_id_as_attribute="true"
  CREATE UNDIRECTED EDGE waypoint (From Trip, To BusStop | From Trip, To TrainStation | From Trip, To Location, trip_seq_no UINT, is_transit_mode BOOL)
  CREATE UNDIRECTED EDGE utilise (From Trip, To BusService | From Trip, To TrainService, from_trip_seq_no UINT, to_trip_seq_no UINT)
""")
print(results)

## Create the Graph

In [ ]:
results = conn.gsql('CREATE GRAPH TigyoreGraph(BusService, BusStop, TrainService, TrainStation, Location, serve, Trip, waypoint, utilise)')

print(results)

In [ ]:
conn.graphname='TigyoreGraph'
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)

conn = tg.TigerGraphConnection(host=hostName, graphname='TigyoreGraph', username=userName, password=password, apiToken=authToken)